In [ ]:
!pip install pytelegrambotapi
!pip install python-telegram-bot
import telebot
import logging
from datetime import datetime
import gspread
from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
import random
import time

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='bot.log'
)
logger = logging.getLogger(__name__)

# Конфигурация
TOKEN = "8160931827:AAHYDPMKOz_4iF0pcLhKlTKmrWnbRkhyxn8"
SHEET_ID = "1qTvWbbHJiDhrGlcuA2YzypDRTQ4MjXEE7aN6f-nGJno"  # Из URL Google Sheets
ADMIN_ID = 1645822796  # Ваш Telegram ID

# Инициализация Google Sheets
try:
    gc = gspread.oauth()  # Авторизация через ваш аккаунт
    sheet = gc.open_by_key("1qTvWbbHJiDhrGlcuA2YzypDRTQ4MjXEE7aN6f-nGJno").sheet1
    logger.info("Успешное подключение к Google Sheets")
except Exception as e:
    logger.error(f"Ошибка подключения к Google Sheets: {e}")
    raise

# Временное хранилище данных
user_data = {}
verification_codes = {}
last_attempt = {}

def start(update: Update, context: CallbackContext) -> None:
    user_id = update.effective_user.id
    chat_id = update.effective_chat.id

    if check_existing_user(user_id):
        update.message.reply_text(
            "Вы уже оставляли свои данные. Для изменения используйте /edit.",
            reply_markup=ReplyKeyboardRemove()
        )
        return

    user_data[user_id] = {'step': 'waiting_for_name'}
    update.message.reply_text(
        "Добрый день! Оставьте контактный номер для связи.\n"
        "Введите ваше имя:",
        reply_markup=ReplyKeyboardRemove()
    )

def handle_message(update: Update, context: CallbackContext) -> None:
    user_id = update.effective_user.id
    text = update.message.text

    if user_id not in user_data:
        update.message.reply_text("Пожалуйста, начните с команды /start")
        return

    current_step = user_data[user_id].get('step')

    if current_step == 'waiting_for_name':
        if not validate_name(text):
            update.message.reply_text("Пожалуйста, введите корректное имя (только буквы, минимум 2 символа)")
            return

        user_data[user_id]['name'] = text
        user_data[user_id]['step'] = 'waiting_for_phone'

        reply_keyboard = [[{"text": "Отправить номер телефона", "request_contact": True}]]
        update.message.reply_text(
            "Теперь введите ваш номер телефона в формате +7XXXXXXXXXX или 8XXXXXXXXXX",
            reply_markup=ReplyKeyboardMarkup(
                reply_keyboard,
                one_time_keyboard=True,
                resize_keyboard=True
            )
        )

    elif current_step == 'waiting_for_phone':
        phone = text.replace(" ", "").replace("-", "").replace("(", "").replace(")", "")

        if not validate_phone(phone):
            update.message.reply_text("Пожалуйста, введите номер в формате +7XXXXXXXXXX или 8XXXXXXXXXX")
            return

        if check_existing_phone(phone):
            update.message.reply_text("Этот номер телефона уже зарегистрирован.")
            del user_data[user_id]
            return

        user_data[user_id]['phone'] = phone
        user_data[user_id]['step'] = 'waiting_for_verification'

        code = str(random.randint(1000, 9999))
        verification_codes[user_id] = code
        last_attempt[user_id] = time.time()

        # В реальном приложении здесь должна быть отправка SMS
        update.message.reply_text(
            f"Код верификации: {code}\n"
            "Введите этот код для подтверждения номера телефона.",
            reply_markup=ReplyKeyboardRemove()
        )

    elif current_step == 'waiting_for_verification':
        if text != verification_codes.get(user_id):
            if time.time() - last_attempt[user_id] < 3600:
                update.message.reply_text("Неверный код. Попробуйте снова через час.")
                return
            last_attempt[user_id] = time.time()
            update.message.reply_text("Неверный код. Пожалуйста, попробуйте еще раз.")
            return

        save_user_data(
            user_id=user_id,
            name=user_data[user_id]['name'],
            phone=user_data[user_id]['phone']
        )

        update.message.reply_text("Благодарю! С вами свяжется наш менеджер.")

        # Очистка временных данных
        clean_user_data(user_id)

def edit_data(update: Update, context: CallbackContext) -> None:
    user_id = update.effective_user.id
    existing = check_existing_user(user_id)

    if not existing:
        update.message.reply_text("Вы еще не оставляли свои данные. Используйте /start.")
        return

    user_data[user_id] = {
        'step': 'waiting_for_name',
        'existing_row': existing['row']
    }
    update.message.reply_text(
        "Введите ваше новое имя или нажмите /cancel для отмены.",
        reply_markup=ReplyKeyboardRemove()
    )

def admin_panel(update: Update, context: CallbackContext) -> None:
    user_id = update.effective_user.id
    if user_id != ADMIN_ID:
        update.message.reply_text("Доступ запрещен.")
        return

    records = sheet.get_all_records()
    count = len(records)

    update.message.reply_text(
        f"📊 Админ-панель\n"
        f"Всего записей: {count}\n"
        f"Последние 5 записей:\n"
        f"{get_last_records(5)}"
    )

# Вспомогательные функции
def validate_name(name):
    return name.replace(" ", "").isalpha() and len(name) >= 2

def validate_phone(phone):
    return (phone.startswith('+7') or phone.startswith('8')) and len(phone) == 12 if phone.startswith('+7') else len(phone) == 11

def check_existing_user(user_id):
    records = sheet.get_all_records()
    for i, row in enumerate(records, start=2):
        if str(row.get('User ID')) == str(user_id):
            return {'row': i, 'data': row}
    return None

def check_existing_phone(phone):
    records = sheet.get_all_records()
    return any(str(row.get('Номер')) == str(phone) for row in records)

def save_user_data(user_id, name, phone):
    try:
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        sheet.append_row([now, user_id, name, phone, "Подтвержден"])
        logger.info(f"Данные сохранены: {user_id}, {name}, {phone}")
    except Exception as e:
        logger.error(f"Ошибка сохранения: {e}")
        raise

def get_last_records(n):
    records = sheet.get_all_records()
    last_records = records[-n:] if len(records) > n else records
    return "\n".join(
        f"{i+1}. {r.get('Имя', '')} - {r.get('Номер', '')} ({r.get('Дата', '')})"
        for i, r in enumerate(last_records)
    )

def clean_user_data(user_id):
    if user_id in user_data:
        del user_data[user_id]
    if user_id in verification_codes:
        del verification_codes[user_id]
    if user_id in last_attempt:
        del last_attempt[user_id]

def error_handler(update: Update, context: CallbackContext):
    logger.error(msg="Ошибка:", exc_info=context.error)
    if update and update.message:
        update.message.reply_text('Произошла ошибка. Пожалуйста, попробуйте позже.')

def main():
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("edit", edit_data))
    dp.add_handler(CommandHandler("admin", admin_panel))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))
    dp.add_error_handler(error_handler)

    logger.info("Бот запущен")
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

ERROR:__main__:Ошибка подключения к Google Sheets: [Errno 2] No such file or directory: '/root/.config/gspread/credentials.json'


FileNotFoundError: [Errno 2] No such file or directory: '/root/.config/gspread/credentials.json'

In [ ]:
import logging
from telegram import Update, KeyboardButton, ReplyKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from twilio.rest import Client
import re
import time

# Настройка логирования
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Настройка Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)
sheet = client.open("Your Google Sheet Name").sheet1

# Настройка Twilio
twilio_client = Client("TWILIO_ACCOUNT_SID", "TWILIO_AUTH_TOKEN")
twilio_number = "YOUR_TWILIO_NUMBER"

# Хранение данных пользователей
user_data = {}

# Функция для проверки имени
def is_valid_name(name):
    return bool(re.match("^[A-Za-zА-Яа-яЁё\s]+$", name))

# Функция для проверки номера телефона
def is_valid_phone(phone):
    return bool(re.match(r"^(?:\+7|8)\d{10}$", phone))

# Команда /start
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Добрый день! Оставьте контактный номер для связи.")
    user_data[update.message.from_user.id] = {}

# Обработка имени
def handle_name(update: Update, context: CallbackContext) -> None:
    name = update.message.text
    if is_valid_name(name):
        user_data[update.message.from_user.id]['name'] = name
        update.message.reply_text("Введите номер телефона в формате +7XXX... или 8XXX...")
    else:
        update.message.reply_text("Пожалуйста, введите корректное имя (только буквы).")

# Обработка номера телефона
def handle_phone(update: Update, context: CallbackContext) -> None:
    phone = update.message.text
    if is_valid_phone(phone):
        user_data[update.message.from_user.id]['phone'] = phone
        # Отправка SMS-кода
        verification_code = "123456"  # Генерация кода
        twilio_client.messages.create(body=f"Ваш код: {verification_code}", from_=twilio_number, to=phone)
        update.message.reply_text("Введите код, отправленный на ваш номер.")
    else:
        update.message.reply_text("Пожалуйста, введите корректный номер телефона.")

# Обработка кода верификации
def handle_verification_code(update: Update, context: CallbackContext) -> None:
    code = update.message.text
    if code == "123456":  # Проверка кода
        user_id = update.message.from_user.id
        name = user_data[user_id]['name']
        phone = user_data[user_id]['phone']
        # Сохранение данных в Google Sheets
        sheet.append_row([time.strftime("%Y-%m-%d %H:%M:%S"), user_id, name, phone, "Verified"])
        update.message.reply_text("Благодарю! С вами свяжется наш менеджер.")
    else:
        update.message.reply_text("Неверный код. Попробуйте еще раз.")

# Команда /admin
def admin(update: Update, context: CallbackContext) -> None:
    # Здесь можно добавить логику для вывода статистики и экспорта в CSV
    update.message.reply_text("Статистика: ...")

# Основная функция
def main() -> None:
    updater = Updater("YOUR_TELEGRAM_BOT_TOKEN")
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_name))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_phone))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_verification_code))
    dispatcher.add_handler(CommandHandler("admin", admin))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()
